In [1]:
gameFolder = "DOA++-MAME"

In [2]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../games',gameFolder))   

tensorBoardFolder = "./ppo2_TB_CustCnn/"
modelFolder = "ppo2_Model_CustCnn/"

os.makedirs(modelFolder, exist_ok=True)

In [3]:
from makeDiambraEnv import *

import tensorflow as tf

from customPolicies.utils import linear_schedule, AutoSave
from customPolicies.customCnnPolicy import *

from stable_baselines import PPO2

In [4]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../../customMAME/"
diambraKwargs["player"] = "Random"
diambraKwargs["frame_ratio"] = 1
diambraKwargs["render"] = False
diambraKwargs["characters"] = ["Kasumi"]

wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"] = [256, 256, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["frame_stack"] = 6
wrapperKwargs["dilation"] = 6
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = 0


#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBufP1") # env.actBufLen*(env.n_actions[0]+env.n_actions[1])
keyToAdd.append("ownHealth")    # 1
keyToAdd.append("oppHealth")    # 1
keyToAdd.append("ownPosition")  # 1
keyToAdd.append("ownPosition")  # 1
keyToAdd.append("stage")        # 1
keyToAdd.append("characters")   # 2*len(env.charNames)

numEnv=8

env = make_diambra_env(diambraMame, env_prefix="Train", num_env=numEnv, seed=timeDepSeed, 
                       continue_game=0, diambra_kwargs=diambraKwargs, 
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd)

envRef = env.envs[0]


Env_id =  Train0
Continue value =  0
1P Environment
Player side = Random , Character = Kasumi
Using frame stacking with dilation =  6
Env_id =  Train1
Continue value =  0
Failed to open, retrying ... 
1P Environment
Player side = Random , Character = Kasumi
Using frame stacking with dilation =  6
Env_id =  Train2
Continue value =  0
Failed to open, retrying ... 
1P Environment
Player side = Random , Character = Kasumi
Using frame stacking with dilation =  6
Env_id =  Train3
Continue value =  0
1P Environment
Player side = Random , Character = Kasumi
Using frame stacking with dilation =  6
Env_id =  Train4
Continue value =  0
Failed to open, retrying ... 
1P Environment
Player side = Random , Character = Kasumi
Using frame stacking with dilation =  6
Env_id =  Train5
Continue value =  0
1P Environment
Player side = Random , Character = Kasumi
Using frame stacking with dilation =  6
Env_id =  Train6
Continue value =  0
Failed to open, retrying ... 
1P Environment
Player side = Random , 

In [5]:
print("Obs_space = ", env.observation_space)
print("Obs_space type = ", env.observation_space.dtype)
print("Obs_space high = ", env.observation_space.high)
print("Obs_space low = ", env.observation_space.low)

Obs_space =  Box(256, 256, 7)
Obs_space type =  float32
Obs_space high =  [[[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 ...

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]

 [[1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  ...
  [1. 1. 1. ... 

In [6]:
print("Act_space = ", env.action_space)
print("Act_space type = ", env.action_space.dtype)
print("Act_space n = ", env.action_space.nvec)

Act_space =  MultiDiscrete([9 8])
Act_space type =  int64
Act_space n =  [9 8]


In [7]:
# Policy param
policyKwargs={}
policyKwargs["n_add_info"] = envRef.actBufLen*(envRef.n_actions[0]+envRef.n_actions[1]) + len(keyToAdd)-2 # No Char Info
policyKwargs["layers"] = [64, 64]

In [8]:
# PPO param
setGamma = 0.94
setLearningRate = linear_schedule(2.5e-4, 2.5e-6)
setClipRange = linear_schedule(0.2, 0.025)
setClipRangeVf = setClipRange

# Initialize the model, 1 env
#model = PPO2(CustCnnPolicy, env, verbose=1, 
#             gamma = setGamma, nminibatches=4, noptepochs=4, n_steps=128,
#             learning_rate=setLearningRate, cliprange=setClipRange, cliprange_vf=setClipRangeVf, 
#             tensorboard_log=tensorBoardFolder, policy_kwargs=policyKwargs)

#OR

# Load the trained agent
model = PPO2.load(modelFolder+"13_5M", env=env, tensorboard_log=tensorBoardFolder, 
                  policy_kwargs=policyKwargs, gamma = setGamma, learning_rate=setLearningRate, 
                  cliprange=setClipRange, cliprange_vf=setClipRangeVf)





Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
print("Model discount factor = ", model.gamma)

In [ ]:
# Create the callback: autosave every USER DEF steps
autoSaveCallback = AutoSave(check_freq=1000000, numEnv=numEnv, save_path=modelFolder+"13_5M_")

# Train the agent
time_steps = 30000000
model.learn(total_timesteps=time_steps, callback=autoSaveCallback)

In [ ]:
# Save the agent
model.save(modelFolder+"33_5M")

## Evaluation

In [ ]:
# Initialize a new evaluation environment
diambraKwargs["render"] = True

env = make_diambra_env(diambraMame, env_prefix="Eval", num_env=1, seed=timeDepSeed, 
                       continue_game=0, diambra_kwargs=diambraKwargs, 
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd)

In [ ]:
observation = env.reset()

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    action = model.predict(observation, deterministic=False)
    #action_prob = model.action_probability(observation, states)
    #print("Action probabilities = ", action_prob)
    #print("Max action = ", np.argmax(action_prob))
    #print("Action = ", action)
    
    observation, reward, done, info = env.step(action[0])
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))   
    
env.close()